Get Chunking

In [3]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import regex as re
import pickle

In [4]:
with open('data/chunks.pkl', 'rb') as file:
    article_list = pickle.load(file)
article_list

['RENEE MONTAGNE, HOST: One of the first things Michelle Obama did as first lady, was to dig up part of the beautifully manicured South Lawn of the White House and plant a garden - a vegetable garden. Today, she\'s out with a book, "American Grown," a kind of diary of that garden through the seasons. We joined her on a hot day last week for a little tour',
 'MICHELLE OBAMA: This is one of my favorite times for the garden, spring, because everything is bursting',
 'MONTAGNE: Tomatoes and tarragon, figs and rhubarb - the stuff of state dinners and family lunches',
 "OBAMA: We have our broccoli here. We eat lots of broccoli, lots of fresh greens and lettuces of all kinds. You know, to my older daughter's unhappiness, snap peas - her least favorite vegetable. And we have so much of it because everybody else loves it. Here, help yourself",
 'MONTAGNE: All right',
 'OBAMA: Taste it',
 'MONTAGNE: That is one of my favorite things, though',
 'OBAMA: Very sweet, which is what I try to tell my d

In [5]:
article_list[0]

'RENEE MONTAGNE, HOST: One of the first things Michelle Obama did as first lady, was to dig up part of the beautifully manicured South Lawn of the White House and plant a garden - a vegetable garden. Today, she\'s out with a book, "American Grown," a kind of diary of that garden through the seasons. We joined her on a hot day last week for a little tour'

Chroma

In [14]:
# Install
%pip install chromadb

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
# Create A chroma Client
import chromadb
chroma_client = chromadb.Client()

In [8]:
# Create a collection
collection = chroma_client.get_or_create_collection(name="my_collection")

In [9]:
# creating id for text doc in collection
id = []
for i in range(len(article_list)):
  id.append("id" + str(i))

In [10]:
# Add text doc to collection
collection.upsert(
    documents = article_list,
    ids = id
)

C:\Users\User\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:22<00:00, 3.71MiB/s]


In [11]:
# query collection
results = collection.query(
    query_texts = ["This article’s political stance is democratic."], # Chroma will embed this for you
    n_results = 5 # how many results to return
)
print(results)

{'ids': [['id24', 'id33', 'id5', 'id11', 'id30']], 'embeddings': None, 'documents': [["OBAMA: It's a little bit of..", 'MONTAGNE: This is NPR News', 'OBAMA: Taste it', "OBAMA: We just can't sell it", 'OBAMA: It is beautiful. I mean, here we are in the heart of Washington, D']], 'uris': None, 'data': None, 'metadatas': [[None, None, None, None, None]], 'distances': [[1.3331794738769531, 1.3858851194381714, 1.4557957649230957, 1.4622066020965576, 1.4712979793548584]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}


In [12]:
# query collection
results = collection.query(
    query_texts = ["This article  is using sensational phrases or words"], # Chroma will embed this for you
    n_results = 5 # how many results to return
)
print(results)

{'ids': [['id5', 'id33', 'id24', 'id7', 'id11']], 'embeddings': None, 'documents': [['OBAMA: Taste it', 'MONTAGNE: This is NPR News', "OBAMA: It's a little bit of..", 'OBAMA: Very sweet, which is what I try to tell my daughter', "OBAMA: We just can't sell it"]], 'uris': None, 'data': None, 'metadatas': [[None, None, None, None, None]], 'distances': [[1.445431113243103, 1.4627740383148193, 1.4928231239318848, 1.5337291955947876, 1.5343693494796753]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}
